In [18]:
import json
from pathlib import Path
import pandas as pd
import hvplot.pandas
import hvplot 

records_path = Path('outputs')
tile_sizes = ['200x200', '400x400', '200x200,400x400']
overlaps = [0, 15, 30]
# exps = list(product(tile_sizes, overlaps))

In [23]:
df_data = {
    'exp': [], 'total_time': [], 'frame_idx': [], 
    'tile_idx': [], 'tile_size': [], 'tile_time': [], 'num_tiles': [], 
    'frac_idx': [], 'frac_preprocessing': [], 'frac_inference': [], 'frac_get_objs': [],
    'postprocessing_time': [], 'num_objs': [], 'objs': [], 'exp_overlap': [], 'exp_tile_sizes': []
}
for exp in records_path.glob('*.json'):
    
    print(exp.name)
    tile_sizes, overlap = exp.name.rstrip('.json').split('_')[1:]
    with exp.open('r') as file:
        data = json.load(file)
    total_time = data['program'][1] - data['program'][0]
    print(f'Total Time: {total_time:.4f}')
    n_frames = len(data['frames'])
    print('Frames:')
    for f_i, frame in enumerate(data['frames']):
        print(f'  Frame = {f_i}')
        n_tiles = len(frame['tiles'])
        all_n_fracs = 0
        for t_i, (tile_size, tiles) in enumerate(zip(tile_sizes.split(','), frame['tiles'])):
            n_fracs = len(tiles['sub'])
            all_n_fracs += n_fracs
            print(f'  Tile size = {tile_size}({n_fracs}) {all_n_fracs}')
            for frac_i, frac in enumerate(tiles['sub']):
                f_preprocessing = frac['end_preprocessing'] - frac['start']
                f_inference = frac['end_inference'] - frac['end_preprocessing']
                f_obj = frac['end_get_object'] - frac['end_preprocessing']
                # print(f'    [{frac_i:03d}] preprocessing={f_preprocessing:.4f} inference={f_inference:.4f} get_objs={f_obj:.4f}')
                # record
                df_data['frac_idx'].append(frac_i)
                df_data['frac_preprocessing'].append(f_preprocessing)
                df_data['frac_inference'].append(f_inference)
                df_data['frac_get_objs'].append(f_obj)
                # ----------
                if frac_i == n_fracs-1:
                    tiles_time = frac['end_get_object'] - tiles['start']
            # record
            df_data['tile_time'] += [tiles_time]*n_fracs
            df_data['tile_idx'] += [t_i]*n_fracs
            df_data['tile_size'] += [tile_size]*n_fracs

            # print(f'  [Tiles Time] {tiles_time:.4f}')
        postprocessing_time = frame['postprocessing'][1] - frame['postprocessing'][0]
        # print(f'  [Post processing] {postprocessing_time:.4f}')        
        # ----------
        # print('  Objects:')
        # print(f"  {list(frame['objects'].keys())}")
        # record
        df_data['num_tiles'] += [n_tiles]*all_n_fracs
        df_data['frame_idx'] += [f_i]*all_n_fracs
        df_data['postprocessing_time'] += [postprocessing_time]*all_n_fracs
        df_data['num_objs'] += [len(frame['objects'])]*all_n_fracs
        df_data['objs'] += [', '.join(list(frame['objects'].keys()))]*all_n_fracs
    df_data['exp_overlap'] += [overlap]*all_n_fracs*n_frames
    df_data['exp_tile_sizes'] += [tile_sizes]*all_n_fracs*n_frames
    df_data['exp'] += [exp.name.rstrip('.json')]*all_n_fracs*n_frames
    df_data['total_time'] += [total_time]*all_n_fracs*n_frames
    # ----------
    print('--'*20)

out_200x200_15.json
Total Time: 160.2671
Frames:
  Frame = 0
  Tile size = 200x200(165) 165
  Frame = 1
  Tile size = 200x200(165) 165
  Frame = 2
  Tile size = 200x200(165) 165
  Frame = 3
  Tile size = 200x200(165) 165
  Frame = 4
  Tile size = 200x200(165) 165
  Frame = 5
  Tile size = 200x200(165) 165
  Frame = 6
  Tile size = 200x200(165) 165
  Frame = 7
  Tile size = 200x200(165) 165
  Frame = 8
  Tile size = 200x200(165) 165
  Frame = 9
  Tile size = 200x200(165) 165
  Frame = 10
  Tile size = 200x200(165) 165
  Frame = 11
  Tile size = 200x200(165) 165
  Frame = 12
  Tile size = 200x200(165) 165
  Frame = 13
  Tile size = 200x200(165) 165
  Frame = 14
  Tile size = 200x200(165) 165
  Frame = 15
  Tile size = 200x200(165) 165
  Frame = 16
  Tile size = 200x200(165) 165
  Frame = 17
  Tile size = 200x200(165) 165
  Frame = 18
  Tile size = 200x200(165) 165
  Frame = 19
  Tile size = 200x200(165) 165
  Frame = 20
  Tile size = 200x200(165) 165
  Frame = 21
  Tile size = 200x200(16

In [24]:
for k, v in df_data.items():
    print(k, len(v))

exp 55080
total_time 55080
frame_idx 55080
tile_idx 55080
tile_size 55080
tile_time 55080
num_tiles 55080
frac_idx 55080
frac_preprocessing 55080
frac_inference 55080
frac_get_objs 55080
postprocessing_time 55080
num_objs 55080
objs 55080
exp_overlap 55080
exp_tile_sizes 55080


In [47]:
df = pd.DataFrame(df_data)
df['exp'] = df['exp'].str.lstrip('out_')

In [48]:
df.to_csv('exp.csv', index=False)

In [49]:
df = pd.read_csv('exp.csv')
df['exp_seperate'] = df['exp_tile_sizes'].astype(str) + '_' + df['tile_size'] + '_' + df['exp_overlap'].astype(str)

In [50]:
df['exp'].unique()

array(['200x200_15', '200x200,400x400_30', '200x200,400x400_0',
       '200x200_0', '400x400_30', '400x400_15', '400x400_0', '200x200_30',
       '200x200,400x400_15'], dtype=object)

In [69]:
df_temp = df.loc[:, ['exp', 'total_time']].drop_duplicates()
df_temp = df_temp.sort_values(by=['exp']).reset_index(drop=True)
df_temp

,exp,total_time
0,"200x200,400x400_0",163.355080
1,"200x200,400x400_15",193.499309
2,"200x200,400x400_30",217.426585
3,200x200_0,134.533258
4,200x200_15,160.267064
5,200x200_30,179.057361
6,400x400_0,66.177721
7,400x400_15,70.980854
8,400x400_30,75.474865


In [79]:
df_temp = df.loc[:, ['exp', 'exp_seperate', 'frame_idx', 'tile_time', 'tile_size']].drop_duplicates()

In [80]:
df_temp.groupby('exp')[['tile_time']].sum()

,tile_time
exp,
"200x200,400x400_0",123.170974
"200x200,400x400_15",152.809567
"200x200,400x400_30",179.075914
200x200_0,96.215099
200x200_15,120.925887
200x200_30,140.749595
400x400_0,28.121162
400x400_15,33.057422
400x400_30,37.734291


In [81]:
df_temp.groupby('exp_seperate')[['tile_time']].agg(['mean', 'std'])

tile_time          
                                mean       std
exp_seperate                                  
200x200,400x400_200x200_0   2.123753  0.022355
200x200,400x400_200x200_15  2.671650  0.020219
200x200,400x400_200x200_30  3.147942  0.018757
200x200,400x400_400x400_0   0.613380  0.010786
200x200,400x400_400x400_15  0.724118  0.008036
200x200,400x400_400x400_30  0.831523  0.007092
200x200_200x200_0           2.138113  0.019790
200x200_200x200_15          2.687242  0.023846
200x200_200x200_30          3.127769  0.028028
400x400_400x400_0           0.624915  0.007969
400x400_400x400_15          0.734609  0.011361
400x400_400x400_30          0.838540  0.012077

In [83]:
df_draw = df_temp.pivot_table(values='tile_time', index='frame_idx', columns='exp_seperate')
df_draw.hvplot.line(x='frame_idx', y=df_draw.columns, height=480, width=720)

:NdOverlay   [exp_seperate]
   :Curve   [frame_idx]   (value)

In [90]:
df_temp = df.loc[:, ['exp', 'exp_seperate', 'frame_idx', 'frac_idx', 'tile_size', 'frac_preprocessing', 'frac_inference', 'frac_get_objs']].drop_duplicates()

In [91]:
df_temp.groupby(['exp', 'tile_size'])[['frac_preprocessing', 'frac_inference', 'frac_get_objs']].agg(['mean', 'std'])

frac_preprocessing           frac_inference  \
                                           mean       std           mean   
exp                tile_size                                               
200x200,400x400_0  200x200             0.003253  0.000461       0.011308   
                   400x400             0.005187  0.000861       0.010636   
200x200,400x400_15 200x200             0.003211  0.000530       0.011235   
                   400x400             0.004631  0.001638       0.010923   
200x200,400x400_30 200x200             0.003114  0.000534       0.011483   
                   400x400             0.004655  0.001347       0.010965   
200x200_0          200x200             0.003317  0.000470       0.011344   
200x200_15         200x200             0.003206  0.000531       0.011286   
200x200_30         200x200             0.003316  0.000549       0.011241   
400x400_0          400x400             0.005318  0.001111       0.010858   
400x400_15         400x400             0.004788  0.001839       0.010993   
400x400_30         400x400             0.004793  0.001536       0.010988   

                                       frac_get_objs            
                                   std          mean       std  
exp                tile_size                                    
200x200,400x400_0  200x200    0.000932      0.013035  0.000989  
                   400x400    0.001431      0.012294  0.001584  
200x200,400x400_15 200x200    0.000980      0.012932  0.001044  
                   400x400    0.001269      0.012560  0.001379  
200x200,400x400_30 200x200    0.000747      0.013233  0.000796  
                   400x400    0.001316      0.012624  0.001442  
200x200_0          200x200    0.000998      0.013082  0.001045  
200x200_15         200x200    0.000918      0.013031  0.000966  
200x200_30         200x200    0.000985      0.012926  0.001024  
400x400_0          400x400    0.001698      0.012486  0.001845  
400x400_15         400x400    0.001598      0.012649  0.001725  
400x400_30         400x400    0.001568      0.012628  0.001669

In [96]:
df_draw1 = df_temp.groupby(['exp_seperate', 'frame_idx'])[['frac_preprocessing']].mean().reset_index().pivot_table(values='frac_preprocessing', index='frame_idx', columns='exp_seperate')
df_draw2 = df_temp.groupby(['exp_seperate', 'frame_idx'])[['frac_inference']].mean().reset_index().pivot_table(values='frac_inference', index='frame_idx', columns='exp_seperate')
df_draw3 = df_temp.groupby(['exp_seperate', 'frame_idx'])[['frac_get_objs']].mean().reset_index().pivot_table(values='frac_get_objs', index='frame_idx', columns='exp_seperate')

p1 = df_draw1.hvplot.line(x='frame_idx', y=df_draw1.columns, height=480, width=720, title='Average Preprocessing Time For Each Fragment')
p2 = df_draw2.hvplot.line(x='frame_idx', y=df_draw2.columns, height=480, width=720, title='Average Inference Time For Each Fragment')
p3 = df_draw3.hvplot.line(x='frame_idx', y=df_draw3.columns, height=480, width=720, title='Average Get Objects Time For Each Fragment')

p1 + p2 + p3

:Layout
   .NdOverlay.I   :NdOverlay   [exp_seperate]
      :Curve   [frame_idx]   (value)
   .NdOverlay.II  :NdOverlay   [exp_seperate]
      :Curve   [frame_idx]   (value)
   .NdOverlay.III :NdOverlay   [exp_seperate]
      :Curve   [frame_idx]   (value)

In [113]:
df_temp = df.loc[:, ['exp', 'frame_idx', 'postprocessing_time', 'num_objs']].drop_duplicates()

In [114]:
df_temp.groupby('exp')[['postprocessing_time', 'num_objs']].mean()

,postprocessing_time,num_objs
exp,,
"200x200,400x400_0",0.014310,16.955556
"200x200,400x400_15",0.021372,18.311111
"200x200,400x400_30",0.011987,21.422222
200x200_0,0.007850,13.577778
200x200_15,0.012954,13.977778
200x200_30,0.008087,17.577778
400x400_0,0.002961,9.577778
400x400_15,0.002801,8.644444
400x400_30,0.002893,9.555556


In [115]:
df_draw = df_temp.pivot_table(values='num_objs', index='frame_idx', columns='exp')
df_draw.hvplot.line(x='frame_idx', y=df_draw.columns, height=480, width=720, title='Number of Object Detected By Each Frame(After Post Processing)')

:NdOverlay   [exp]
   :Curve   [frame_idx]   (value)

In [116]:
df_temp = df.loc[:, ['exp', 'frame_idx', 'postprocessing_time', 'objs']].drop_duplicates()

In [117]:
df_temp['objs_list'] = df_temp['objs'].str.split(', ')

In [119]:
df_temp.dropna().groupby('exp')['objs_list']

In [128]:
objs = {}
for exp in df_temp['exp'].unique():
    objs[exp] = set()
    df_temp2 = df_temp.loc[df_temp['exp'] == exp, 'objs_list'].dropna()
    for v in df_temp2.values:
        objs[exp].update(v)

In [135]:
for k, v in objs.items():
    print(f'{k}: {len(v)}')
    print(f'{v}')
    print()

200x200_15: 36
{'train', 'carrot', 'sports ball', 'hot dog', 'cell phone', 'toothbrush', 'tv', 'banana', 'chair', 'dog', 'book', 'bed', 'boat', 'handbag', 'traffic light', 'car', 'cat', 'zebra', 'bus', 'orange', 'apple', 'donut', 'vase', 'airplane', 'mouse', 'remote', 'bench', 'bird', 'person', 'bottle', 'tie', 'toilet', 'umbrella', 'cake', 'suitcase', 'fire hydrant'}

200x200,400x400_30: 42
{'train', 'carrot', 'laptop', 'refrigerator', 'teddy bear', 'snowboard', 'sports ball', 'hot dog', 'cell phone', 'toothbrush', 'banana', 'tv', 'chair', 'dog', 'book', 'bed', 'knife', 'motorcycle', 'cup', 'traffic light', 'car', 'zebra', 'bus', 'orange', 'apple', 'donut', 'vase', 'airplane', 'mouse', 'potted plant', 'remote', 'bench', 'kite', 'bird', 'person', 'bottle', 'tie', 'surfboard', 'umbrella', 'cake', 'suitcase', 'fire hydrant'}

200x200,400x400_0: 35
{'train', 'carrot', 'laptop', 'refrigerator', 'teddy bear', 'snowboard', 'hot dog', 'toothbrush', 'tv', 'banana', 'chair', 'dog', 'book', 'bed